In [39]:
### Question A.1

In [40]:
from pyspark.sql import SparkSession

In [41]:
spark_session = SparkSession.builder.master("spark://192.168.2.122:7077").appName("kulathunga-vidumini").config("spark.dynamicAllocation.enabled", True).config("spark.dynamicAllocation.shuffleTracking.enabled",True).config("spark.shuffle.service.enabled", True).config("spark.dynamicAllocation.executorIdleTimeout","30s").config("spark.cores.max", 2).getOrCreate()


In [42]:
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [43]:
def add(a, b):
    # associative and commutative! 
    return a + b
rdd = spark_session.sparkContext.parallelize(range(10**7))
result = rdd.filter(lambda x: x % 2 == 0).map(lambda x: x ** 2).reduce(add) 
print(result)

166666616666670000000


In [44]:
lines = spark_context.textFile('hdfs://192.168.2.122:9000/europarl/europarl-v7.sv-en.en')
swedish_lines = spark_context.textFile('hdfs://192.168.2.122:9000/europarl/europarl-v7.sv-en.sv')
lines.first()

'Resumption of the session'

In [45]:
lines.count()

1862234

In [46]:
swedish_lines.count()

1862234

In [47]:
lines.getNumPartitions()

2

In [48]:
swedish_lines.getNumPartitions()

3

In [49]:
### Question A.2

In [50]:
def preprocess(lines, flatten):
    # lowercase the text
    lowercase = lines.map(lambda line: line.lower())
    # tokenize the text
    if flatten:
        words = lowercase.flatMap(lambda line: line.split(' '))
    else:
        words = lowercase.map(lambda line: line.split(' '))
    return words

words_en = preprocess(lines, False)
words_sv = preprocess(swedish_lines, False)

In [10]:
words_en.take(10)

[['resumption', 'of', 'the', 'session'],
 ['i',
  'declare',
  'resumed',
  'the',
  'session',
  'of',
  'the',
  'european',
  'parliament',
  'adjourned',
  'on',
  'friday',
  '17',
  'december',
  '1999,',
  'and',
  'i',
  'would',
  'like',
  'once',
  'again',
  'to',
  'wish',
  'you',
  'a',
  'happy',
  'new',
  'year',
  'in',
  'the',
  'hope',
  'that',
  'you',
  'enjoyed',
  'a',
  'pleasant',
  'festive',
  'period.'],
 ['although,',
  'as',
  'you',
  'will',
  'have',
  'seen,',
  'the',
  'dreaded',
  "'millennium",
  "bug'",
  'failed',
  'to',
  'materialise,',
  'still',
  'the',
  'people',
  'in',
  'a',
  'number',
  'of',
  'countries',
  'suffered',
  'a',
  'series',
  'of',
  'natural',
  'disasters',
  'that',
  'truly',
  'were',
  'dreadful.'],
 ['you',
  'have',
  'requested',
  'a',
  'debate',
  'on',
  'this',
  'subject',
  'in',
  'the',
  'course',
  'of',
  'the',
  'next',
  'few',
  'days,',
  'during',
  'this',
  'part-session.'],
 ['in',
  

In [51]:
words_sv.take(10)

[['återupptagande', 'av', 'sessionen'],
 ['jag',
  'förklarar',
  'europaparlamentets',
  'session',
  'återupptagen',
  'efter',
  'avbrottet',
  'den',
  '17',
  'december.',
  'jag',
  'vill',
  'på',
  'nytt',
  'önska',
  'er',
  'ett',
  'gott',
  'nytt',
  'år',
  'och',
  'jag',
  'hoppas',
  'att',
  'ni',
  'haft',
  'en',
  'trevlig',
  'semester.'],
 ['som',
  'ni',
  'kunnat',
  'konstatera',
  'ägde',
  '"den',
  'stora',
  'år',
  '2000-buggen"',
  'aldrig',
  'rum.',
  'däremot',
  'har',
  'invånarna',
  'i',
  'ett',
  'antal',
  'av',
  'våra',
  'medlemsländer',
  'drabbats',
  'av',
  'naturkatastrofer',
  'som',
  'verkligen',
  'varit',
  'förskräckliga.'],
 ['ni',
  'har',
  'begärt',
  'en',
  'debatt',
  'i',
  'ämnet',
  'under',
  'sammanträdesperiodens',
  'kommande',
  'dagar.'],
 ['till',
  'dess',
  'vill',
  'jag',
  'att',
  'vi,',
  'som',
  'ett',
  'antal',
  'kolleger',
  'begärt,',
  'håller',
  'en',
  'tyst',
  'minut',
  'för',
  'offren',
  'f

In [52]:
words_en.count()

1862234

In [53]:
words_sv.count()

1862234

In [54]:
### Question A.3

In [55]:
words_en_flat = preprocess(lines, True)
words_sv_flat = preprocess(swedish_lines, True)

In [56]:
unique_values_en = words_en_flat.countByValue()
unique_values_sv = words_sv_flat.countByValue()

In [57]:
unique_values_en

defaultdict(int,
            {'resumption': 523,
             'of': 1659758,
             'the': 3498375,
             'session': 2371,
             'i': 501393,
             'declare': 1360,
             'resumed': 1165,
             'european': 268599,
             'parliament': 73330,
             'adjourned': 359,
             'on': 465854,
             'friday': 560,
             '17': 1718,
             'december': 4341,
             '1999,': 1035,
             'and': 1288401,
             'would': 153260,
             'like': 93105,
             'once': 15643,
             'again': 14509,
             'to': 1539760,
             'wish': 17034,
             'you': 105208,
             'a': 773522,
             'happy': 3763,
             'new': 67237,
             'year': 16870,
             'in': 1085993,
             'hope': 26282,
             'that': 797516,
             'enjoyed': 764,
             'pleasant': 232,
             'festive': 40,
             'period.': 1480,
  

In [58]:
res = {key: val for key, val in sorted(unique_values_en.items(), key = lambda ele: ele[1])}
for x in list(reversed(list(res)))[0:10]:
    print (x, res[x])

the 3498375
of 1659758
to 1539760
and 1288401
in 1085993
that 797516
a 773522
is 758050
for 534242
we 522849


In [59]:
res_sv = {key: val for key, val in sorted(unique_values_sv.items(), key = lambda ele: ele[1])}
for x in list(reversed(list(res_sv)))[0:10]:
    print (x, res_sv[x])

att 1706293
och 1344830
i 1050774
det 924866
som 913276
för 908680
av 738068
är 694381
en 620310
vi 539797


In [60]:
### Question A.4

In [61]:
en1 = lines.zipWithIndex()

In [62]:
en1.take(10)

[('Resumption of the session', 0),
 ('I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
  1),
 ("Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
  2),
 ('You have requested a debate on this subject in the course of the next few days, during this part-session.',
  3),
 ("In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.",
  4),
 ("Please rise, then, for this minute' s silence.", 5),
 ("(The House rose and observed a minute' s silence)", 6),
 ('Madam President, on a point of order.', 7),
 ('You will be aware

In [63]:
sv1 = swedish_lines.zipWithIndex()

In [64]:
# Swap the key and value - so that the line number is the key.
swap_sv1 = sv1.map(lambda x: (x[1], x[0]))
swap_en1 = en1.map(lambda x: (x[1], x[0]))

In [65]:
# Join the two RDDs together according to the line number key
new_rdd = swap_en1.join(swap_sv1)

In [66]:
#Filter to exclude line pairs that have an empty/missing “corresponding” sentence.
filtered_rdd2 = new_rdd.filter(lambda x: x[1][0] != "" and x[1][1] != "")

In [67]:
# Filter to leave only pairs of sentences with a small number of words per sentence (filltering senteces less than 10 words)
filtered_rdd3 = filtered_rdd2.filter(lambda x: len(x[1][0].split()) <= 10 and len(x[1][1].split()) <= 10)

In [68]:
#Filter to leave only pairs of sentences with the same number of words in each sentence
filtered_rdd4 = filtered_rdd3.filter(lambda x: len(x[1][0].split()) == len(x[1][1].split()))

In [69]:
def LAS2(line):
    # Unpacking the tuple into two sentences
    sentence1, sentence2 = line[1]
    # Creating a list of word pairs by zipping together the lowercased and split words from each sentence
    result = list(zip(sentence1.lower().split(), sentence2.lower().split()))
    # Return the list of word pairs
    return result

In [70]:
filtered_rdd5 = filtered_rdd4.flatMap(LAS2)

In [71]:
filtered_rdd6 = filtered_rdd5.map(lambda word: (word,1))

In [72]:
filtered_group = filtered_rdd6.groupByKey()
words_outout = filtered_group.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)

In [73]:
words_outout.take(10)

[(11250, ('is', 'är')),
 (6604, ('we', 'vi')),
 (6142, ('i', 'jag')),
 (3941, ('and', 'och')),
 (3785, ('this', 'detta')),
 (3505, ('a', 'en')),
 (3418, ('it', 'det')),
 (3097, ('not', 'inte')),
 (3044, ('that', 'det')),
 (2980, ('closed.', 'avslutad.'))]

In [74]:
spark_session.stop()